In [1]:
import re
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Function to parse the log file (unchanged from original)
def parse_log_file(log_file_path):
    epochs = []
    top1_accuracies = []
    top5_accuracies = []
    learning_rates = []

    curr_epoch = None
    last_lr_for_epoch = None

    with open(log_file_path, 'r') as f:
        for line in f:
            # Parse training epoch learning rate
            train_lr_match = re.search(r"Epoch\(train\)\s*\[(\d+)\].*?lr: ([0-9\.e+-]+)", line)
            if train_lr_match:
                curr_epoch = int(train_lr_match.group(1))
                last_lr_for_epoch = float(train_lr_match.group(2))
                continue

            # Parse validation accuracy at end of val epoch
            val_acc_match = re.search(r"Epoch\(val\) \[(\d+)\].*accuracy/top1: ([0-9\.]+)\s+accuracy/top5: ([0-9\.]+)", line)
            if val_acc_match:
                epoch = int(val_acc_match.group(1))
                top1 = float(val_acc_match.group(2))
                top5 = float(val_acc_match.group(3))
                # Save values
                epochs.append(epoch)
                top1_accuracies.append(top1)
                top5_accuracies.append(top5)
                # Save learning rate for the epoch
                learning_rates.append(last_lr_for_epoch)

    return epochs, top1_accuracies, top5_accuracies, learning_rates

# New function to plot interactively with Plotly
def plot_accuracies_and_lr_interactive(epochs, top1, top5, lrs, x_range):
    # Create a figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add Top-1 Accuracy trace
    fig.add_trace(
        go.Scatter(x=epochs, y=top1, name='Top-1 Accuracy', line=dict(color='blue')),
        secondary_y=False,
    )

    # Add Top-5 Accuracy trace
    fig.add_trace(
        go.Scatter(x=epochs, y=top5, name='Top-5 Accuracy', line=dict(color='orange')),
        secondary_y=False,
    )

    # Add Learning Rate trace on secondary y-axis
    fig.add_trace(
        go.Scatter(x=epochs, y=lrs, name='Learning Rate', line=dict(color='green')),
        secondary_y=True,
    )

    # Update layout for interactivity and axes
    fig.update_layout(
        title='Validation Accuracy and Training Learning Rate vs Epoch',
        xaxis_title='Epoch',
        yaxis_title='Accuracy (%)',
        yaxis2_title='Learning Rate',
        hovermode='x unified',  # Vertical crosshair on x-hover, shows values for all lines
        legend=dict(yanchor='top', y=1.15, xanchor='right', x=1.15),  # Top-right legend
    )

    # Set x-axis ranges
    fig.update_xaxes(range=x_range)
    
    # Set y-axis ranges
    fig.update_yaxes(range=[0, 100], dtick=10, secondary_y=False)
    fig.update_yaxes(secondary_y=True)  # Auto-range for LR

    # Show the figure (interactive in Jupyter)
    fig.show()


In [2]:
adam_range = [0, 310]
sgd_range = [0, 110]

In [ ]:
# SEBNet Adam, LR 1e-03
log_file_path = 'path/to/output.log'
epochs, top1, top5, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, top1, top5, lrs, adam_range)